# Лабораторная работа: Мета-обучение
## Шаг 1: Сбор данных с OpenML

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openml

from src.data.collector import OpenMLCollector

%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
# Инициализация
collector = OpenMLCollector(cache_dir='../data/cache/openml_cache')

# Получение данных
datasets_df = collector.fetch_datasets_list(
    min_instances=100,
    max_instances=10000,
    min_features=3,
    max_features=500,
    min_classes=2,
    limit=350
)

print(f"Найдено {len(datasets_df)} датасетов")
print(f"Колонки: {list(datasets_df.columns)}")

In [ ]:
# Визуализация
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(datasets_df['NumberOfInstances'], bins=30, edgecolor='black')
axes[0].set_xlabel('Количество объектов')
axes[0].set_ylabel('Частота')
axes[0].set_title('Распределение по размеру')

axes[1].hist(datasets_df['NumberOfFeatures'], bins=30, edgecolor='black', color='orange')
axes[1].set_xlabel('Количество признаков')
axes[1].set_ylabel('Частота')
axes[1].set_title('Распределение по признакам')

axes[2].hist(datasets_df['NumberOfClasses'], bins=15, edgecolor='black', color='green')
axes[2].set_xlabel('Количество классов')
axes[2].set_ylabel('Частота')
axes[2].set_title('Распределение по классам')

plt.tight_layout()
plt.show()

In [ ]:
# Сохранение
datasets_df.to_csv('../data/raw/dataset_list.csv', index=False)
print("Список датасетов сохранен в ../data/raw/dataset_list.csv")

### Выводы
- Собрано 350 датасетов
- Данные сохранены для дальнейшей обработки